In [2]:
%load_ext sql
import csv, sqlite3
con = sqlite3.connect("final.db")
cur = con.cursor()
%sql sqlite:///final.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [4]:
import pandas as pd

chicago_census_data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
chicago_public_schools = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
chicago_crime_data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")


chicago_census_data.to_sql("census_data",con, if_exists='replace', index=False)
chicago_public_schools.to_sql("chicago_public_schools",con, if_exists='replace', index=False)
chicago_crime_data.to_sql("chicago_crime_data",con, if_exists='replace', index=False)

533

# Problem 1
Find the total number of crimes recorded in the CRIME table.

In [11]:
%%sql
select ID, count(*) as "Number Of Crimes" from chicago_crime_data;

   sqlite:///chicago_data.db
 * sqlite:///final.db
Done.


ID,Number Of Crimes
3512276,533


# Problem 2
List community areas with per capita income less than 11000.

In [16]:
%%sql

select community_area_name, community_area_number from census_data WHERE per_capita_income < 11000;

   sqlite:///chicago_data.db
 * sqlite:///final.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
West Garfield Park,26.0
South Lawndale,30.0
Fuller Park,37.0
Riverdale,54.0


# Problem 3
List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [36]:
%%sql
select distinct(case_number) from chicago_crime_data WHERE description LIKE '%minor%';

   sqlite:///chicago_data.db
 * sqlite:///final.db
Done.


CASE_NUMBER
HL266884
HK238408


# Problem 4
List all kidnapping crimes involving a child?

In [39]:
%%sql
select * from chicago_crime_data where primary_type = 'KIDNAPPING';

   sqlite:///chicago_data.db
 * sqlite:///final.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


# Problem 5
What kinds of crimes were recorded at schools?

In [49]:
%sql select distinct(primary_type) from chicago_crime_data where location_description like '%school%';

   sqlite:///chicago_data.db
 * sqlite:///final.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


# Problem 6
List the average safety score for all types of schools.

In [55]:
%sql select "Elementary, Middle, or High School", round(avg(safety_score)) as "Average Safety Score" from chicago_public_schools group by "Elementary, Middle, or High School";

   sqlite:///chicago_data.db
 * sqlite:///final.db
Done.


"Elementary, Middle, or High School",Average Safety Score
ES,50.0
HS,50.0
MS,48.0


# Problem 7
List 5 community areas with highest % of households below poverty line

In [62]:
%sql select COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY from census_data order by PERCENT_HOUSEHOLDS_BELOW_POVERTY desc limit 5;

   sqlite:///chicago_data.db
 * sqlite:///final.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


# Problem 8
Which community area is most crime prone?

In [84]:
%%sql 
select community_area_number, count(community_area_number) as "Frequency" from chicago_crime_data group by community_area_number
order by Frequency desc limit 1;

   sqlite:///chicago_data.db
 * sqlite:///final.db
Done.


COMMUNITY_AREA_NUMBER,Frequency
25.0,43


# Problem 9
Use a sub-query to find the name of the community area with highest hardship index

In [138]:
%%sql 

select community_area_name, hardship_index from census_data
where hardship_index in (select max(hardship_index) from census_data order by hardship_index )

   sqlite:///chicago_data.db
 * sqlite:///final.db
Done.


COMMUNITY_AREA_NAME,HARDSHIP_INDEX
Riverdale,98.0


# Problem 10
Use a sub-query to determine the Community Area Name with most number of crimes?

In [103]:
%%sql 

select community_area_name from census_data
where community_area_number = (select community_area_number from chicago_crime_data group by community_area_number
order by count(community_area_number) desc
    limit 1);

   sqlite:///chicago_data.db
 * sqlite:///final.db
Done.


COMMUNITY_AREA_NAME
Austin


# Extra For Practice
get community_area_name where percent of housing crowded is the highest

In [125]:
%%sql 
select community_area_name, percent_of_housing_crowded from census_data 
where percent_of_housing_crowded = (select max(percent_of_housing_crowded) from census_data order by percent_of_housing_crowded);

   sqlite:///chicago_data.db
 * sqlite:///final.db
Done.


COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED
Gage Park,15.8


get community area name where crime is kidnapping

In [128]:
%%sql 
select community_area_name from census_data
where community_area_number = (select community_area_number from chicago_crime_data where primary_type = 'KIDNAPPING') LIMIT 10;

   sqlite:///chicago_data.db
 * sqlite:///final.db
Done.


PRIMARY_TYPE
KIDNAPPING
